In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


In [8]:
url = 'https://mydramalist.com/shows/top?page=1'



In [18]:
def initialize_browser():
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    print('Starting Driver')
    title_element = driver.find_element(by=By.CSS_SELECTOR, value='h6.title a')
    title_element.click()

initialize_browser()

'''
this thing from gpt:
    # Wait for the title element to be clickable
    title_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "h6.title a"))
    )
'''    

Starting Driver


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the Chrome webdriver
driver = webdriver.Chrome()

# Navigate to the webpage
driver.get("https://mydramalist.com/shows/top?page=1")

try:
    # Wait for the title element to be clickable
    title_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "h6.title a"))
    )

    # Click on the title of the drama
    title_element.click()

    # You can perform other actions on the new page here if needed

except Exception as e:
    print(e)

finally:
    # Close the browser window
    driver.quit()


KeyboardInterrupt: 